In [1]:
import pandas as pd
import numpy as np
from dtaidistance import dtw
import os
from sklearn.cluster import SpectralClustering
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'/home/maxime/Documents/WORKSPACES/forecasting_models/cluster/features_clustering'

In [3]:
data = pd.read_csv('../../tests/enc_X_train.csv')

In [4]:
data

,nb_emmergencies_CHU Dijon,O3_FR26005,O3_FR26010,PM10_FR26005,PM10_FR26014,PM25_FR26005,NO2_FR26005,NO2_FR26014,inc_grippe,inc_diarrhee,...,confinement1,confinement2,couvrefeux,ramadan,before_HNFC_moving,during_HNFC_moving,after_HNFC_moving,before_COVID,during_COVID,after_COVID
0,1.431246,-0.103467,-0.075270,-0.011981,-0.065712,0.240391,0.598583,0.237403,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
1,1.195802,-0.688671,-0.371901,0.260223,-0.197136,-0.014353,1.903848,1.344561,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
2,1.431246,-1.524676,-1.431300,0.940733,1.117106,1.480146,1.903848,1.221543,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
3,0.934198,-1.817278,-1.939811,3.118367,3.351317,4.197416,2.164900,1.344561,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
4,1.091160,-1.106674,-0.838037,1.893448,1.905651,2.668952,1.251215,0.237403,3.023719,-0.144729,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,0.515631,0.523537,0.348490,1.212938,1.774227,0.630999,0.729109,1.590596,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1149,1.221962,0.857939,0.645121,0.668529,0.591409,0.070562,-0.745839,-0.008631,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1150,0.986518,0.356335,0.348490,-0.011981,0.065712,0.410220,0.337530,1.467578,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1151,-0.033739,0.021933,0.094234,-0.148083,-0.459985,0.240391,-0.419523,0.360421,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1


In [5]:
# Calcul de la matrice de distance DTW
def calculate_dtw_distance_matrix(df):
    num_series = df.shape[1]
    dist_matrix = np.zeros((num_series, num_series))
    
    for i in range(num_series):
        for j in range(i + 1, num_series):
            # Calcul de la distance DTW entre chaque paire de séries
            dist = dtw.distance(df.iloc[:, i], df.iloc[:, j])
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist  # La matrice est symétrique

    return pd.DataFrame(dist_matrix, columns=df.columns, index=df.columns)

In [6]:
dtw_matrix = calculate_dtw_distance_matrix(data)
dtw_matrix

,nb_emmergencies_CHU Dijon,O3_FR26005,O3_FR26010,PM10_FR26005,PM10_FR26014,PM25_FR26005,NO2_FR26005,NO2_FR26014,inc_grippe,inc_diarrhee,...,confinement1,confinement2,couvrefeux,ramadan,before_HNFC_moving,during_HNFC_moving,after_HNFC_moving,before_COVID,during_COVID,after_COVID
nb_emmergencies_CHU Dijon,0.000000,18.854103,18.378808,23.160712,23.655629,24.303761,21.575700,19.788295,28.943993,23.956220,...,33.162748,33.196083,33.196341,31.547236,33.955854,33.955854,48.020829,32.951519,33.150743,33.525863
O3_FR26005,18.854103,0.000000,8.892888,24.055128,23.092166,22.949970,21.694762,19.070596,27.899957,24.390950,...,31.541035,31.542449,31.542868,29.437077,33.955854,33.955854,48.020829,33.150187,31.543083,34.008551
O3_FR26010,18.378808,8.892888,0.000000,23.621973,22.898917,22.768274,21.447306,18.757795,27.619180,23.927404,...,32.007520,32.007641,32.007757,29.699632,33.955854,33.955854,48.020829,33.804959,32.009353,34.123069
PM10_FR26005,23.160712,24.055128,23.621973,0.000000,11.206357,14.893167,19.272275,20.617208,28.459999,25.940655,...,32.959619,32.958718,32.966579,31.659245,33.955854,33.955854,48.020829,33.276086,32.993258,33.623302
PM10_FR26014,23.655629,23.092166,22.898917,11.206357,0.000000,14.824701,19.238754,20.397577,28.647540,25.665227,...,32.929490,32.915168,32.910063,31.841011,33.955854,33.955854,48.020829,33.871951,32.911623,33.623871
PM25_FR26005,24.303761,22.949970,22.768274,14.893167,14.824701,0.000000,17.362051,19.278860,25.268951,24.195850,...,32.805023,32.805000,32.805992,30.846061,33.955854,33.955854,48.020829,33.009316,32.812562,33.182282
NO2_FR26005,21.575700,21.694762,21.447306,19.272275,19.238754,17.362051,0.000000,13.737213,26.049320,25.098194,...,32.445656,32.466019,32.470335,30.700496,33.955854,33.955854,48.020829,32.091722,32.442430,33.772640
NO2_FR26014,19.788295,19.070596,18.757795,20.617208,20.397577,19.278860,13.737213,0.000000,26.563267,24.473526,...,31.992946,31.993209,31.994087,30.419564,33.955854,33.955854,48.020829,33.038172,31.998413,33.694594
inc_grippe,28.943993,27.899957,27.619180,28.459999,28.647540,25.268951,26.049320,26.563267,0.000000,25.646352,...,30.141559,30.198283,30.221294,28.153202,33.955854,33.955854,48.020829,31.456990,30.169202,33.873289
inc_diarrhee,23.956220,24.390950,23.927404,25.940655,25.665227,24.195850,25.098194,24.473526,25.646352,0.000000,...,30.247026,30.247034,30.296281,28.109462,33.955854,33.955854,48.020829,35.191070,30.478331,32.521743


In [13]:
dtw_matrix.to_csv("dtw_matrix.csv")

In [7]:
def distance_to_similarity_gaussian(distance_matrix, sigma=1.0):
    # Transformation en similarité avec une fonction gaussienne
    similarity_matrix = np.exp(-distance_matrix**2 / (2 * sigma**2))
    np.fill_diagonal(similarity_matrix.values, 1)  # Similitude maximale sur la diagonale
    return similarity_matrix

In [8]:
similarity_matrix_gaussian = distance_to_similarity_gaussian(dtw_matrix)
similarity_matrix_gaussian

,nb_emmergencies_CHU Dijon,O3_FR26005,O3_FR26010,PM10_FR26005,PM10_FR26014,PM25_FR26005,NO2_FR26005,NO2_FR26014,inc_grippe,inc_diarrhee,...,confinement1,confinement2,couvrefeux,ramadan,before_HNFC_moving,during_HNFC_moving,after_HNFC_moving,before_COVID,during_COVID,after_COVID
nb_emmergencies_CHU Dijon,1.000000e+00,6.443266e-78,4.486218e-74,3.297526e-117,3.067867e-122,5.458001e-129,8.234008e-102,9.337157e-86,1.213045e-182,2.393858e-125,...,1.543264e-239,5.106207e-240,5.062711e-240,7.743869e-217,4.258250e-251,4.258250e-251,0.000000e+00,1.663276e-236,2.297844e-239,8.513208e-245
O3_FR26005,6.443266e-78,1.000000e+00,6.718105e-18,2.228056e-126,1.609305e-116,4.249131e-115,6.264565e-103,1.062275e-79,9.352811e-170,6.532873e-130,...,9.417181e-217,9.006399e-217,8.887926e-217,6.806208e-189,4.258250e-251,4.258250e-251,0.000000e+00,2.340544e-239,8.827905e-217,7.104008e-252
O3_FR26010,4.486218e-74,6.718105e-18,1.000000e+00,6.797603e-122,1.369571e-114,2.704629e-113,1.303488e-100,3.941726e-77,2.269560e-166,4.772223e-125,...,3.441109e-223,3.427831e-223,3.415153e-223,2.892882e-192,4.258250e-251,4.258250e-251,0.000000e+00,7.071392e-249,3.245034e-223,1.436358e-253
PM10_FR26005,3.297526e-117,2.228056e-126,6.797603e-122,1.000000e+00,5.371791e-28,6.844590e-49,2.223561e-81,4.981791e-93,1.308940e-176,7.547588e-147,...,1.273577e-236,1.311965e-236,1.012509e-236,2.247108e-218,4.258250e-251,4.258250e-251,0.000000e+00,3.575314e-241,4.200184e-237,3.230719e-246
PM10_FR26014,3.067867e-122,1.609305e-116,1.369571e-114,5.371791e-28,1.000000e+00,1.893091e-48,4.239988e-81,4.502630e-91,6.183961e-179,9.210321e-144,...,3.436455e-236,5.506511e-236,6.514111e-236,7.002738e-221,4.258250e-251,4.258250e-251,0.000000e+00,7.328281e-250,6.188010e-236,3.169579e-246
PM25_FR26005,5.458001e-129,4.249131e-115,2.704629e-113,6.844590e-49,1.893091e-48,1.000000e+00,3.491043e-66,1.958484e-81,2.224196e-139,7.473116e-128,...,2.054757e-234,2.056307e-234,1.990455e-234,2.448282e-207,4.258250e-251,4.258250e-251,0.000000e+00,2.472447e-237,1.604497e-234,8.072842e-240
NO2_FR26005,8.234008e-102,6.264565e-103,1.303488e-100,2.223561e-81,4.239988e-81,3.491043e-66,1.000000e+00,1.051776e-41,4.477439e-148,1.639668e-137,...,2.538805e-229,1.310991e-229,1.139568e-229,2.159182e-205,4.258250e-251,4.258250e-251,0.000000e+00,2.315816e-224,2.818933e-229,2.107515e-248
NO2_FR26014,9.337157e-86,1.062275e-79,3.941726e-77,4.981791e-93,4.502630e-91,1.958484e-81,1.051776e-41,1.000000e+00,6.016592e-154,8.687708e-131,...,5.485920e-223,5.439929e-223,5.289161e-223,1.155643e-201,4.258250e-251,4.258250e-251,0.000000e+00,9.533954e-238,4.605481e-223,2.932030e-247
inc_grippe,1.213045e-182,9.352811e-170,2.269560e-166,1.308940e-176,6.183961e-179,2.224196e-139,4.477439e-148,6.016592e-154,1.000000e+00,1.494797e-143,...,5.233310e-198,9.452490e-199,4.716737e-199,7.735590e-173,4.258250e-251,4.258250e-251,0.000000e+00,1.329351e-215,2.273852e-198,7.003618e-250
inc_diarrhee,2.393858e-125,6.532873e-130,4.772223e-125,7.547588e-147,9.210321e-144,7.473116e-128,1.639668e-137,8.687708e-131,1.494797e-143,1.000000e+00,...,2.166592e-199,2.166057e-199,4.877793e-200,2.647740e-172,4.258250e-251,4.258250e-251,0.000000e+00,1.208875e-269,1.930674e-202,2.144061e-230


In [9]:
def clustering_with_spectral(similarity_matrix, n_clusters=3):
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')
    labels = spectral.fit_predict(similarity_matrix)
    return labels

In [10]:
labels_spectral = clustering_with_spectral(similarity_matrix_gaussian)
labels_spectral

/home/maxime/Documents/WORKSPACES/forecasting_models/.venv/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:301: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/maxime/Documents/WORKSPACES/forecasting_models/.venv/lib/python3.10/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [11]:
def tsne_visualization(similarity_matrix):
    # Conversion de la matrice de similarité en matrice de distance
    distance_matrix = 1 - similarity_matrix
    tsne = TSNE(metric='precomputed', n_components=2, perplexity=30)
    tsne_result = tsne.fit_transform(distance_matrix)
    return tsne_result

In [12]:
# Visualisation t-SNE
tsne_result = tsne_visualization(similarity_matrix_gaussian)
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels)  # `labels` peut venir du clustering
plt.title('t-SNE Visualisation based on Similarity Matrix')
plt.show()

ValueError: The parameter init="pca" cannot be used with metric="precomputed".